In [2]:
import google.generativeai as genai
import pandas as pd
import time
import numpy as np
from scipy import stats
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error, mean_absolute_error

C:\Users\fabio\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [6]:
manual_df = pd.read_csv("data/manual_annotated_dataset/manual_data_v2.csv")
cleaned_df = pd.read_csv("data/cleaned_dataset/merged.tsv", sep=",", encoding="utf-8")

merged_df = manual_df.merge(cleaned_df, how='left', left_on='id', right_on="ID")

#Rimuovo i testi usati nel few shot prompt
prompt_text = ["ParlaMint-IT_2014-03-26-LEG17-Senato-sed-217.u153", "ParlaMint-IT_2013-06-25-LEG17-Senato-sed-50.u53", "ParlaMint-IT_2018-06-05-LEG18-Senato-sed-9.u62","ParlaMint-IT_2021-06-09-LEG18-Senato-sed-334.u46"]
ground_df = merged_df.drop(merged_df[merged_df['id'].isin(prompt_text)].index)

In [9]:
#Estraggo i testi da dare in input a gemini
texts_to_classify = []
for id in ground_df["id"]:
    texts_to_classify.append((id, ground_df[ground_df["id"] == id].text.iloc[0]))

### GEN AI

In [ ]:
API_KEY = ""
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel('gemini-2.0-flash')

In [37]:
def classify_record(text_id, text, prompt, model):
    filled_prompt = prompt.replace('[$$$]',text)
    res = model.generate_content(filled_prompt)
    time.sleep(4)
    return str(text_id + "," +res.text)

In [44]:
#CALCOLO CLASSIFICAZIONI GEMINI
def calcola_classificazioni(texts_to_classify, prompt):
    i = 0
    gemini_classes = []
    for tupla in texts_to_classify:
        res = classify_record(tupla[0], tupla[1], prompt, model)
        gemini_classes.append(res)
        i+=1
        print(i, end = " ")
        
    classification_list = []
    for stringa in gemini_classes:
        stringa = stringa.strip("\n")
        lista = stringa.split(",")
        lista = [lista[0]] + [int(x) for x in lista[1:]]
        classification_list.append(lista)

    classification_df = pd.DataFrame(classification_list, columns = ['id', 'manichean', 'indivisible', 'peoplecentrism', 'antielitism', 'simplicism', 'emotional'])
    return classification_df

In [45]:
def calcola_prestazioni(ground_df, classification_df):
    dimensions = ['manichean', 'indivisible', 'peoplecentrism', 'antielitism', 'simplicism', 'emotional']
    for dim in dimensions:
        print("ANALISI: ", dim)
        print("SPEARMAN: ", stats.spearmanr(ground_df[dim], classification_df[dim]))
        print("MSE: ", mean_squared_error(ground_df[dim], classification_df[dim]))
        print("MAE: ", mean_absolute_error(ground_df[dim], classification_df[dim]))
        print(classification_report(ground_df[dim], classification_df[dim]))
        print("\n\n")

### Few Shot

In [46]:
with open('genai/v2/fewshot_6features_prompt.txt', 'r', encoding="utf8") as f:
    prompt = f.read()

In [47]:
classification_df = pd.read_csv("genai/v2/fewshot_6features_classifications.csv", sep=",", encoding="utf-8")
#classification_df = calcola_classificazioni(texts_to_classify, prompt)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 

In [49]:
classification_df.to_csv("genai/v2/fewshot_6features_classifications.csv", index=False)

In [50]:
calcola_prestazioni(ground_df, classification_df)

ANALISI:  manichean
SPEARMAN:  SignificanceResult(statistic=0.8002008986387475, pvalue=3.1120905511061726e-12)
MSE:  0.1
MAE:  0.1
              precision    recall  f1-score   support

           0       0.87      0.96      0.92        28
           1       0.95      0.82      0.88        22

    accuracy                           0.90        50
   macro avg       0.91      0.89      0.90        50
weighted avg       0.90      0.90      0.90        50




ANALISI:  indivisible
SPEARMAN:  SignificanceResult(statistic=0.4745734709147427, pvalue=0.0004977039759165544)
MSE:  0.12
MAE:  0.12
              precision    recall  f1-score   support

           0       0.89      0.98      0.93        42
           1       0.75      0.38      0.50         8

    accuracy                           0.88        50
   macro avg       0.82      0.68      0.72        50
weighted avg       0.87      0.88      0.86        50




ANALISI:  peoplecentrism
SPEARMAN:  SignificanceResult(statistic=0.59677636

### Zero Shot

In [57]:
with open('genai/v2/zeroshot_6features_prompt.txt', 'r', encoding="utf8") as f:
    zero_shot_prompt = f.read()

In [60]:
#zero_shot_classification = pd.read_csv("genai/zero-shot_50_texts_classifications.csv", sep=",", encoding="utf-8")
zero_shot_classification = calcola_classificazioni(texts_to_classify, zero_shot_prompt)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 

In [62]:
zero_shot_classification.to_csv("genai/v2/zeroshot_6features_classifications.csv", index=False)

In [63]:
calcola_prestazioni(ground_df, zero_shot_classification)

ANALISI:  manichean
SPEARMAN:  SignificanceResult(statistic=0.5942944359687179, pvalue=5.358675824345657e-06)
MSE:  0.2
MAE:  0.2
              precision    recall  f1-score   support

           0       0.78      0.89      0.83        28
           1       0.83      0.68      0.75        22

    accuracy                           0.80        50
   macro avg       0.81      0.79      0.79        50
weighted avg       0.80      0.80      0.80        50




ANALISI:  indivisible
SPEARMAN:  SignificanceResult(statistic=nan, pvalue=nan)
MSE:  0.16
MAE:  0.16
              precision    recall  f1-score   support

           0       0.84      1.00      0.91        42
           1       0.00      0.00      0.00         8

    accuracy                           0.84        50
   macro avg       0.42      0.50      0.46        50
weighted avg       0.71      0.84      0.77        50




ANALISI:  peoplecentrism
SPEARMAN:  SignificanceResult(statistic=0.726106525690946, pvalue=2.4107939361602424

C:\Users\fabio\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:4916: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\fabio\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined an